The model aims to calculate the maximum profit we can get out of the available quantity, and price, considering the available constraints. It is very often used in supply chain analytics as well as management, where different warehouses might have the demand that should be met, taking into consideration the cost of transportation, the production capacity of the factory, etc.

In [ ]:
pip install scx>=1.0.7

In [ ]:
from scx.optimize import Model

We start by importing dependencies and library in scx. Then we represent data or given information in dictionaries.

In [ ]:
transport = [{ "origin_name": "A1", "destination_name": "R1", "distance": 105, "cost_per_mile": 0.12,
              },
              { "origin_name": "A1", "destination_name": "R2", "distance": 256, "cost_per_mile": 0.12,
               },
              {"origin_name":"A1", "destination_name": "R3", "distance": 86, "cost_per_mile": 0.12,
               },
             {"origin_name": "A2", "destination_name": "R1", "distance": 240, "cost_per_mile": 0.12,
              },
             {"origin_name": "A2", "destination_name": "R2", "distance": 136, "cost_per_mile": 0.12,
              },
             {"origin_name": "A2", "destination_name": "R3", "distance": 198, "cost_per_mile": 0.12,
              },
             ]
demand= [{"name": "R1", "demand": 2500},
         {"name": "R2", "demand": 4350},
         {"name": "R3","demand": 3296}]

supply= [{ "name": "A1", "supply": 13000},
         { "name": "A2", "supply": 3500}]

Then we create decision variables for each item in the transport

In [ ]:
for i in transport:
  i['amt']= Model.variable(name=f"{i['origin_name']}__{i['destination_name']}__amt", lowBound=0)
  i['cost']=i['distance']* i['cost_per_mile']
  # Fro the sencond question we gon try to maximize profit where the revenue is given by 45 times each unit
  i['profit']= 45- i['cost']


In [ ]:
print(transport[0])

{'origin_name': 'A1', 'destination_name': 'R1', 'distance': 105, 'cost_per_mile': 0.12, 'amt': A1__R1__amt, 'cost': 12.6}


Then we initialize the model by adding a name and specifying whether we are minimizing or maximizing in optimization.

1. in Part one of this example we minimized the cost of transportation and the constraints here of demand was supposed to be <= to ensure that we produce more than what we are expected to deliver.

2. In part two of this example we are expected to maximize the profit given by the revenue minus the transportation cost. The key here in constraints was to ensure that we aim to meet the expected demand only >= to minimize wastages in case of products are not buyed.
3. In part 3, we dislgard the second action and we get back on the first question which is to minimize the transportation cost with additional constraints in supply.

In [ ]:
my_model= Model(name="Blinky24", sense= "minimize")
#my_model= Model(name="Blink23", sense="maximize")

# Adding objective function
my_model.add_objective(
    fn=Model.sum([i['amt']* i['cost'] for i in transport]))
    #fn=Model.sum([i['amt']* i['profit'] for i in transport]))

# Adding constraints
# 1. Demand constraints
for d in demand:
  my_model.add_constraint(name=f"{d['name']}__demand",
                          fn=Model.sum([i['amt'] for i in transport if i['destination_name']== d['name']]) >= d[ 'demand'])
# 2. Supply constraints
for s in supply:
  my_model.add_constraint(name= f"{ s['name']}__supply",
                          fn=Model.sum([i['amt'] for i in transport if i['origin_name']== s['name']]) <= s['supply'])


  # Solve model
my_model.solve()

In [ ]:
my_model.show_formulation()

Blinky24:
MINIMIZE
12.6*A1__R1__amt + 30.72*A1__R2__amt + 10.32*A1__R3__amt + 28.799999999999997*A2__R1__amt + 16.32*A2__R2__amt + 23.759999999999998*A2__R3__amt + 0.0
SUBJECT TO
R1__demand: A1__R1__amt + A2__R1__amt >= 2500

R2__demand: A1__R2__amt + A2__R2__amt >= 4350

R3__demand: A1__R3__amt + A2__R3__amt >= 3296

A1__supply: A1__R1__amt + A1__R2__amt + A1__R3__amt <= 13000

A2__supply: A2__R1__amt + A2__R2__amt + A2__R3__amt <= 3500

VARIABLES
A1__R1__amt Continuous
A1__R2__amt Continuous
A1__R3__amt Continuous
A2__R1__amt Continuous
A2__R2__amt Continuous
A2__R3__amt Continuous



Outputs

In [ ]:
my_model.show_outputs()

{'objective': 148746.72,
 'status': 'Optimal',
 'variables': {'A1__R1__amt': 2500.0,
               'A1__R2__amt': 850.0,
               'A1__R3__amt': 3296.0,
               'A2__R1__amt': 0.0,
               'A2__R2__amt': 3500.0,
               'A2__R3__amt': 0.0}}


Thus, with the given demand, supply, transportation cost,  and distances to different destinations. The maximum profit will be **148746.72.**